<a href="https://colab.research.google.com/github/Nikoschenk/language_model_finetuning/blob/master/scibert_summaries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Running Bert Extractive Summarizer

In [0]:
!pip install bert-extractive-summarizer==0.4.2

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# Path to your CORD-19 fine-tuned SciBERT model checkpoints.
!ls '/content/drive/My Drive/finetuned_models/SciBERT-finetuned/'

In [5]:
from summarizer import Summarizer

import torch
import time
from transformers import *

assert torch.cuda.is_available()

# Specifcy path or models names:
# 1. SciBert (orignal model by Allen AI, https://github.com/allenai/scibert)
# 2. SciBert (your fine-tuned SciBert model on CORD-19 data)
# 3. Bert base model
BERTS = ['allenai/scibert_scivocab_uncased',
         '/content/drive/My Drive/finetuned_models/SciBERT-finetuned/checkpoint-1000',
         'bert-base-uncased']

# Text which you would like to summarize.
body_text = '''
A number of cases with Viral Pneumonia of Unknown Etiology (VPUE) were admitted in Wuhan hospital, on 29th December 2019. These cases were reported to the China Center for Disease Control (CDC). Investigators found that the patients had previously worked in the Huanan Seafood Wholesale Market, a selling hub of animals. The samples drawn from these individuals were sent for laboratory analyses. Later, the etiology was confirmed to be a novel β-genus coronavirus (2019-nCoV). The disease was, hence, termed novel coronavirus-infected pneumonia (NCIP) [1].
According to WHO Coronavirus Disease 2019 (COVID-19) situation report of 17 February 2020, 71,429 cases have been documented globally [2]. Out of these, the number of cases from China is 70,635 and the rest 794 cases are from twenty-four different countries including Japan, Republic of Korea, Vietnam, Singapore, Australia, Malaysia, Cambodia, Philippines, Thailand, Nepal, India, Sri Lanka, United States of America, Canada, France, Germany, Italy, the United Kingdom, Russian Federation, Belgium, Finland, Spain, Sweden, and the United Arab Emirates. The disease has claimed 1775 victims so far, the fatality rate is 2.8% [2,3].
Coronavirus belongs to the family of Coronavirinae, order Nidovirales [4]. They are large, positive-sense RNA viruses comprising of four genera: alpha, beta, delta, and gamma [5]. Until 2019, only six human coronaviruses (HCoVs) were known that could be held responsible for respiratory diseases. Out of these, four (HCoV–229E, OC43, NL63, and HKU1) are endemic globally and are the causative agents for 10%–30% of the upper respiratory infections [4,5]. The other two, Severe Acute Respiratory Syndrome Coronavirus (SARS-CoV) and Middle East respiratory syndrome coronavirus (MERS-CoV), are known to infect the lower respiratory tract [4]. These viruses possess the ability to propagate at an alarming rate, thereby resulting in disease clusters from a single index patient [6].
In 2002, a pandemic of SARS occurred endangering the lives of 8098 individuals with a mortality of 774 patients [5]. The epicenter of the disease was Guangdong, China, from where it spread internationally to more than twelve countries. It was assumed that bats were the natural hosts. Mechanical ventilation support was given to 20%–30% of the infected patients and the fatality rate was 10%. The human-to-human airborne transmission was perceived, and the pandemic was culminated by employing strict public health measures [5]. In 2012, a rapid-spreading infection, MERS, caused by MERS-CoV, emerged in Saudi Arabia. Bats were considered to be the natural hosts and the intermediate host was dromedary camels. A total of 2494 cases with 858 deaths were reported to be caused by expeditious nosocomial transmission. MERS demonstrated clinical features similar to SARS with prominent gastrointestinal symptoms and acute kidney failure. Mechanical ventilation was given to 50%–89% of patients, associated with a mortality rate of 36% [5].
An epidemiological investigation of 198 early cases revealed that only 22% of patients had direct exposure to the marketplace, 32% were in contact with the suspected cases and 51% had no contact with either of the source [1]. However, the virus was capable of efficient human-to-human transmission and similar to MERS, reports of nosocomial propagation were also documented [1]. This situation necessitated the need for the implementation of measures to abstain transmissions such as fit-tested N95 respirator and other personal protective equipment. Furthermore, respiratory samples should be immediately sent for analysis, if a diagnosis is suspected, serum antibodies should be examined before and after exposure to 2019-nCoV [7].
A study conducted in a cluster of recent pneumonia cases in Jin Yin-tan hospital, Wuhan, portrayed the common symptoms were fever, cough, dyspnea, and pneumonia with findings of bilateral ground-glass appearance on chest CT scans [7]. These symptoms were consistent with those of SARS and MERS. The complications of the illness included acute respiratory distress syndrome (ARDS), RNAaemia, acute cardiac injury, and secondary infection. During the study, 32% of the patients were admitted in ICU and 15% died, indicating a significant association with high mortality [7].
Confounding similarities are obtained between 2019-nCoV and SARS-CoV. Prominent amino acid homology is shown between the two by genomic sequence analyses [5]. The clinical manifestations of both viruses were comparable too, with ARDS being developed and oxygen therapy required in both illnesses [7]. This depicts the potential of 2019-nCoV to cause a pandemic similar to SARS which has a history of taxing the global economy $30 to $100 billion.5 Based on this experience, the economic burden of this virus is temporarily estimated to be 1%–2% of GDP (Gross Domestic Product). However, more than 20% of the world GDP now is contributed by China, as compared to 7.5% in 2013 with having substantial proportion in global value chains [8].
In 2017, SARS-CoV and MERS-CoV were added by WHO on its Priority Pathogen list. The diagnostic tests available were rapidly updated to identify 2019-nCoV, allowing early recognition and detection of the cases.5 Preclinical investigations recognize the potent efficacy of Remdesivir against MERS and SARS, which can also be used against 2019-nCoV. Randomized controlled trials are being conducted to evaluate the potential of a combination of lopinavir and ritonavir which showed promising effect against MERS-CoV to treat infected 2019-nCoV patients [7]. Similarly, vaccines are also being developed for SARS and MERS. The nucleic acid vaccine approach is being utilized by the National Institute of Allergy and Infectious Diseases Vaccine Research Center. Research is being conducted to target the messenger RNA (mRNA) vaccine technology and assembling viral vectors to develop a vaccine [5].
The epidemic increases by twofold in 6.4 days [6]. The lengthy incubation period makes the isolation of suspected cases difficult. Therefore, counter-measures like school shutdown, exit screenings, and dissolution of mass gatherings are necessary [1,9]. SARS pandemic ended by employing stringent public health measures. WHO has developed strategic objectives for this response such as limiting human-to-human transmission and zoonotic transmission, early detection and management of victims, gathering data to identify unknown clinical aspects of this disease, sharing event information to all communities, minimizing social and economic impact via multisectoral partnerships and two online courses on an open learning platform to provide real-time knowledge [2,5].
The source of zoonotic infection has not been identified. However, pangolins, a delicacy in Chinese cuisine are suspected, contributing to a ban of game consumption by Chinese government [10]. Under the International Health Regulations 2005, an emergency committee on 2019-nCoV has put Public Health Emergency of International Concern (PHEIC) into effect [11]. For the states with weaker health systems, a US$675 million preparedness and response plan has been launched for the period of February-April 2020 [12]. A disease commodity package has been prepared for patients with necessary biomedical equipment [2]. The impact of the outbreak, on a global scale, remains to be seen.
'''

SUMMARY_RATIO = 0.1

# Evaluate custom models.
for BERT_PATH in BERTS:

  # Load model, model config and tokenizer via Transformers
  custom_config = AutoConfig.from_pretrained(BERT_PATH)
  custom_config.output_hidden_states=True
  custom_tokenizer = AutoTokenizer.from_pretrained(BERT_PATH)
  custom_model = AutoModel.from_pretrained(BERT_PATH, config=custom_config)
  model = Summarizer(custom_model=custom_model, custom_tokenizer=custom_tokenizer)

  assert model.model.model.device.type == 'cuda'

  start = time.time()
  resp = model(body_text, ratio=SUMMARY_RATIO)
  end = time.time()
  print('Model type:', BERT_PATH)
  print(f'Response Time: {end-start}')
  # TODO: Split into sentences and pretty-print.
  print('Summary: ', resp, "\n")




Model type: allenai/scibert_scivocab_uncased
Response Time: 0.6699740886688232
Summary:  A number of cases with Viral Pneumonia of Unknown Etiology (VPUE) were admitted in Wuhan hospital, on 29th December 2019. The human-to-human airborne transmission was perceived, and the pandemic was culminated by employing strict public health measures [5]. This depicts the potential of 2019-nCoV to cause a pandemic similar to SARS which has a history of taxing the global economy $30 to $100 billion.5 Based on this experience, the economic burden of this virus is temporarily estimated to be 1%–2% of GDP (Gross Domestic Product). Similarly, vaccines are also being developed for SARS and MERS. The lengthy incubation period makes the isolation of suspected cases difficult. Under the International Health Regulations 2005, an emergency committee on 2019-nCoV has put Public Health Emergency of International Concern (PHEIC) into effect [11]. 

Model type: /content/drive/My Drive/finetuned_models/SciBERT-f